In [1]:
path_to_json_file = 'Reddit_Twitter_dataset/twitter/sarcasm_detection_shared_task_twitter_training.jsonl'

In [2]:
import json 
di = dict()

with open(path_to_json_file, 'r') as j:
     json_data = [json.loads(line) for line in j]

i=0
for tweet in json_data:
    tot_cont=""
    for con in tweet['context']:
      tot_cont+=(con + " ")
    tot_cont+=tweet['response']
    lab = 0
    if tweet['label'] == 'SARCASM':
        lab=1
    d1 = {'text': tot_cont, 'label': lab}

    di[i] = d1
    i+=1

In [3]:
print(type(di))
print(di[0])

<class 'dict'>
{'text': "A minor child deserves privacy and should be kept out of politics . Pamela Karlan , you should be ashamed of your very angry and obviously biased public pandering , and using a child to do it .@USER If your child isn't named Barron ... #BeBest Melania couldn't care less . Fact . 💯@USER @USER @USER I don't get this .. obviously you do care or you would've moved right along .. instead you decided to care and troll her ..", 'label': 1}


In [4]:
from transformers import AutoTokenizer
from transformers import CanineTokenizer, CanineForSequenceClassification

tokenizer = CanineTokenizer.from_pretrained("google/canine-s")

tokenized_datasets = []
for i in di:
  l = tokenizer(di[i]["text"], padding="max_length", truncation=True)
  l['labels'] = di[i]["label"]
  tokenized_datasets.append(l)


Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


In [5]:
import random
import numpy as np
import torch 
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [6]:
len(tokenized_datasets)

5000

In [7]:
random.shuffle(tokenized_datasets)

train_data = tokenized_datasets[:3000]
test_data = tokenized_datasets[3000:4000]
val_data = tokenized_datasets[4000:]

In [8]:
#train_data[0]

{'input_ids': [57344, 71, 111, 111, 100, 32, 116, 111, 32, 115, 101, 101, 32, 116, 104, 101, 114, 101, 32, 97, 114, 101, 32, 115, 111, 109, 101, 32, 105, 110, 116, 101, 108, 108, 105, 103, 101, 110, 116, 32, 105, 110, 102, 108, 117, 101, 110, 99, 101, 115, 32, 64, 85, 83, 69, 82, 32, 105, 110, 32, 116, 104, 101, 32, 100, 101, 97, 100, 32, 98, 121, 32, 100, 97, 121, 108, 105, 103, 104, 116, 32, 99, 111, 109, 109, 117, 110, 105, 116, 121, 32, 119, 104, 111, 32, 99, 97, 110, 32, 97, 99, 116, 117, 97, 108, 121, 32, 104, 111, 108, 100, 32, 97, 32, 99, 111, 110, 118, 101, 114, 115, 97, 116, 105, 111, 110, 32, 46, 32, 85, 110, 116, 105, 108, 108, 32, 116, 111, 100, 97, 121, 32, 44, 32, 73, 32, 106, 117, 115, 116, 32, 115, 97, 119, 32, 105, 109, 98, 101, 99, 105, 108, 101, 115, 32, 112, 97, 115, 115, 105, 118, 101, 108, 121, 32, 97, 103, 103, 114, 101, 115, 115, 105, 118, 101, 108, 121, 32, 109, 111, 99, 107, 105, 110, 103, 32, 97, 110, 100, 32, 98, 101, 108, 105, 116, 116, 108, 105, 110, 103,

In [9]:
import torch

model = CanineForSequenceClassification.from_pretrained("google/canine-s")#, problem_type="multi_label_classification")




Some weights of CanineForSequenceClassification were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer/3ep_runs")

In [11]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer/3ep_runs", 
                                  evaluation_strategy="epoch")

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,
)

In [15]:
trainer.train()
trainer.save_model("test_trainer")

/home/mss9240/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.694956,0.494000
2,0.689500,0.625054,0.683000
3,0.612000,0.570681,0.703000


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to test_trainer/3ep_runs/checkpoint-500
Configuration saved in test_trainer/3ep_runs/checkpoint-500/config.json
Model weights saved in test_trainer/3ep_runs/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to test_trainer/3ep_runs/checkpoint-1000
Configuration saved in test_trainer/3ep_runs/checkpoint-1000/config.json
Model weights saved in test_trainer/3ep_runs/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to test_trainer
Configuration saved in test_trainer/config.json
Model weights saved in test_trainer/pytorch_model.bin


In [16]:
trainer.evaluate(eval_dataset = test_data)

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'eval_loss': 0.5467970967292786,
 'eval_accuracy': 0.743,
 'eval_runtime': 26.4897,
 'eval_samples_per_second': 37.751,
 'eval_steps_per_second': 4.719,
 'epoch': 3.0}